Deadline: 24 novembre ore 23:59.
Solve efficiently a generic n^2-1 puzzle using path search algorithm.

Cost=  total number of actions you need to __evaluate__. An action is something that bring me to a new state. For example the number of swaps to do.

The result is the sequence of action that took you at the end. The goal is not to find a state but a sequence of actions from srtarting point to end point: we do not look for a soluzion but for a sequence of actions.

In [8]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np

In [9]:
PUZZLE_DIM = 4
action = namedtuple('Action', ['pos1', 'pos2'])

In [10]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

The state is a numpy array.

We created a function that returns the number of actions from a state pos1 to a state pos2.

Compute 100_000 random actions:

In [11]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing: 100%|██████████| 100000/100000 [00:00<00:00, 166093.35it/s]


array([[11,  2,  0,  4],
       [ 9, 15, 13,  6],
       [ 7,  1,  8, 12],
       [10,  3, 14,  5]])

Define a function that indicates if we end the search.

In [12]:
def test_goal(solution):
    arr_solution = np.reshape(solution, PUZZLE_DIM*PUZZLE_DIM)
    arr_solution_no_zero = arr_solution[0: len(arr_solution)-1]
    if np.all(arr_solution_no_zero[:-1] <= arr_solution_no_zero[1:]) and arr_solution[len(arr_solution)-1]==0:
        return True
    return False

# Depth search


In [13]:
# from collections import namedtuple, deque
# import numpy as np

# # Define action as a named tuple
# action = namedtuple('Action', ['pos1', 'pos2'])

# def depth_limited_search(initial_state: np.ndarray, final_state: np.ndarray, max_depth: int) -> list[action] or None:
#     """
#     Performs a depth-limited search to solve the n-puzzle.
    
#     Parameters:
#     - initial_state (np.ndarray): The starting state of the puzzle.
#     - final_state (np.ndarray): The desired goal state of the puzzle.
#     - max_depth (int): The maximum depth limit for the search.
    
#     Returns:
#     - list[action] or None: A sequence of actions leading to the solution, or None if no solution is found.
#     """
#     stack = deque([(initial_state, [], 0)])  # Stack of (current state, path to reach it, current depth)
#     visited = set()  # Set of visited states for current path only
    
#     while stack:
#         current_state, path, depth = stack.pop()
        
#         # Check if we reached the goal
#         if np.array_equal(current_state, final_state):
#             return path
        
#         # Backtrack if depth limit reached
#         if depth >= max_depth:
#             continue
        
#         # Add current state to visited set (track only in current path)
#         visited.add(current_state.tobytes())
        
#         # Generate and iterate over all possible moves
#         for act in available_actions(current_state):
#             next_state = do_action(current_state, act)
            
#             # Check if the next state has already been visited in the current path
#             if next_state.tobytes() not in visited:
#                 # Add the new state and path to stack, increase depth
#                 stack.append((next_state, path + [act], depth + 1))
        
#         # Remove the current state from visited set after backtracking
#         visited.remove(current_state.tobytes())
    
#     return None  # Return None if no solution is found within depth limit

# # Iterative Deepening Depth-First Search (IDDFS) wrapper function
# def iterative_deepening_dfs(initial_state: np.ndarray, final_state: np.ndarray, max_depth: int = 50) -> list[action] or None:
#     """
#     Performs an iterative deepening DFS to solve the n-puzzle.
    
#     Parameters:
#     - initial_state (np.ndarray): The starting state of the puzzle.
#     - final_state (np.ndarray): The desired goal state of the puzzle.
#     - max_depth (int): The maximum depth to limit the search for IDDFS.
    
#     Returns:
#     - list[action] or None: A sequence of actions leading to the solution, or None if no solution is found.
#     """
#     for depth in range(1, max_depth + 1):
#         result = depth_limited_search(initial_state, final_state, depth)
#         if result is not None:
#             return result
#     return None


# test_goal = np.arange(1, PUZZLE_DIM*PUZZLE_DIM, 1)
# test_goal = np.append(test_goal, 0)
# test_goal = test_goal.reshape((PUZZLE_DIM, PUZZLE_DIM))
# iterative_deepening_dfs(state, test_goal)

In [14]:
# A*:
from heapq import heappop, heappush
from collections import namedtuple
import numpy as np

# Define action as a named tuple
action = namedtuple('Action', ['pos1', 'pos2'])

def manhattan_distance(state: np.ndarray, goal_state: np.ndarray) -> int:
    """
    Calculates the Manhattan distance for each tile in the puzzle.
    """
    distance = 0
    for value in range(1, state.size):  # Skip 0 (empty space)
        target_pos = np.argwhere(goal_state == value)[0]
        current_pos = np.argwhere(state == value)[0]
        distance += abs(target_pos[0] - current_pos[0]) + abs(target_pos[1] - current_pos[1])
    return distance

def a_star_search(initial_state: np.ndarray, final_state: np.ndarray) -> list[action] or None:
    """
    Performs an A* search to solve the n-puzzle.
    
    Parameters:
    - initial_state (np.ndarray): The starting state of the puzzle.
    - final_state (np.ndarray): The desired goal state of the puzzle.
    
    Returns:
    - list[action] or None: A sequence of actions leading to the solution, or None if no solution is found.
    """
    # Priority queue: (f_score, g_score, current_state, path)
    open_set = []
    heappush(open_set, (manhattan_distance(initial_state, final_state), 0, initial_state.tobytes(), []))
    #this function push into open_set a new element continuing to garantee the heap properties.
    # the element that we insert are 4.
    
    visited = set()  
    
    while open_set:
        f_score, g_score, current_bytes, path = heappop(open_set)
        current_state = np.frombuffer(current_bytes, dtype=initial_state.dtype).reshape(initial_state.shape)
        
        # Check if we reached the goal
        if np.array_equal(current_state, final_state):
            return path
        
        # Mark current state as visited
        visited.add(current_bytes)
        
        # Generate and iterate over all possible moves
        for act in available_actions(current_state):
            next_state = do_action(current_state, act)
            next_bytes = next_state.tobytes()
            if next_bytes in visited:
                continue
            
            # Calculate scores for the next state
            new_g_score = g_score + 1
            new_f_score = new_g_score + manhattan_distance(next_state, final_state)
            
            # Push the new state into the priority queue with updated scores
            heappush(open_set, (new_f_score, new_g_score, next_bytes, path + [act]))
    
    return None  # Return None if no solution is found


test_goal = np.arange(1, PUZZLE_DIM*PUZZLE_DIM, 1)
test_goal = np.append(test_goal, 0)
test_goal = test_goal.reshape((PUZZLE_DIM, PUZZLE_DIM))
#joblib
res=a_star_search(state, test_goal)
res

[Action(pos1=(0, 2), pos2=(0, 1)),
 Action(pos1=(0, 1), pos2=(1, 1)),
 Action(pos1=(1, 1), pos2=(2, 1)),
 Action(pos1=(2, 1), pos2=(2, 0)),
 Action(pos1=(2, 0), pos2=(1, 0)),
 Action(pos1=(1, 0), pos2=(1, 1)),
 Action(pos1=(1, 1), pos2=(0, 1)),
 Action(pos1=(0, 1), pos2=(0, 0)),
 Action(pos1=(0, 0), pos2=(1, 0)),
 Action(pos1=(1, 0), pos2=(2, 0)),
 Action(pos1=(2, 0), pos2=(2, 1)),
 Action(pos1=(2, 1), pos2=(3, 1)),
 Action(pos1=(3, 1), pos2=(3, 2)),
 Action(pos1=(3, 2), pos2=(3, 3)),
 Action(pos1=(3, 3), pos2=(2, 3)),
 Action(pos1=(2, 3), pos2=(2, 2)),
 Action(pos1=(2, 2), pos2=(1, 2)),
 Action(pos1=(1, 2), pos2=(1, 1)),
 Action(pos1=(1, 1), pos2=(2, 1)),
 Action(pos1=(2, 1), pos2=(2, 2)),
 Action(pos1=(2, 2), pos2=(3, 2)),
 Action(pos1=(3, 2), pos2=(3, 1)),
 Action(pos1=(3, 1), pos2=(2, 1)),
 Action(pos1=(2, 1), pos2=(2, 2)),
 Action(pos1=(2, 2), pos2=(1, 2)),
 Action(pos1=(1, 2), pos2=(1, 1)),
 Action(pos1=(1, 1), pos2=(0, 1)),
 Action(pos1=(0, 1), pos2=(0, 2)),
 Action(pos1=(0, 2),

Breath first ti fa trovare una soluzione ottima. Depth la prima soluzione ottima.

In [21]:
import random
import itertools
import collections
import time

class Node:
    """
    A class representing an Solver node
    - 'puzzle' is a Puzzle instance
    - 'parent' is the preceding node generated by the solver, if any
    - 'action' is the action taken to produce puzzle, if any
    """
    def __init__(self, puzzle, parent=None, action=None):
        self.puzzle = puzzle
        self.parent = parent
        self.action = action
        if (self.parent != None):
            self.g = parent.g + 1
        else:
            self.g = 0

    @property
    def score(self):
        return (self.g + self.h)

    @property
    def state(self):
        """
        Return a hashable representation of self
        """
        return str(self)

    @property 
    def path(self):
        """
        Reconstruct a path from to the root 'parent'
        """
        node, p = self, []
        while node:
            p.append(node)
            node = node.parent
        yield from reversed(p)

    @property
    def solved(self):
        """ Wrapper to check if 'puzzle' is solved """
        return self.puzzle.solved

    @property
    def actions(self):
        """ Wrapper for 'actions' accessible at current state """
        return self.puzzle.actions

    @property
    def h(self):
        """"h"""
        return self.puzzle.manhattan

    @property
    def f(self):
        """"f"""
        return self.h + self.g

    def __str__(self):
        return str(self.puzzle)

class Solver:
    """
    An '8-puzzle' solver
    - 'start' is a Puzzle instance
    """
    def __init__(self, start):
        self.start = start

    def solve(self):
        """
        Perform breadth first search and return a path
        to the solution, if it exists
        """
        queue = collections.deque([Node(self.start)])
        seen = set()
        seen.add(queue[0].state)
        while queue:
            queue = collections.deque(sorted(list(queue), key=lambda node: node.f))
            node = queue.popleft()
            if node.solved:
                return node.path

            for move, action in node.actions:
                child = Node(move(), node, action)

                if child.state not in seen:
                    queue.appendleft(child)
                    seen.add(child.state)

class Puzzle:
    """
    A class representing an '8-puzzle'.
    - 'board' should be a square list of lists with integer entries 0...width^2 - 1
       e.g. [[1,2,3],[4,0,6],[7,5,8]]
    """
    def __init__(self, board):
        self.width = len(board[0])
        self.board = board

    @property
    def solved(self):
        """
        The puzzle is solved if the flattened board's numbers are in
        increasing order from left to right and the '0' tile is in the
        last position on the board
        """
        N = self.width * self.width
        return str(self) == ''.join(map(str, range(1,N))) + '0'

    @property 
    def actions(self):
        """
        Return a list of 'move', 'action' pairs. 'move' can be called
        to return a new puzzle that results in sliding the '0' tile in
        the direction of 'action'.
        """
        def create_move(at, to):
            return lambda: self._move(at, to)

        moves = []
        for i, j in itertools.product(range(self.width),
                                      range(self.width)):
            direcs = {'R':(i, j-1),
                      'L':(i, j+1),
                      'D':(i-1, j),
                      'U':(i+1, j)}

            for action, (r, c) in direcs.items():
                if r >= 0 and c >= 0 and r < self.width and c < self.width and \
                   self.board[r][c] == 0:
                    move = create_move((i,j), (r,c)), action
                    moves.append(move)
        return moves

    @property
    def manhattan(self):
        distance = 0
        for i in range(3):
            for j in range(3):
                if self.board[i][j] != 0:
                    x, y = divmod(self.board[i][j]-1, 3)
                    distance += abs(x - i) + abs(y - j)
        return distance

    def shuffle(self):
        """
        Return a new puzzle that has been shuffled with 1000 random moves
        """
        puzzle = self
        for _ in range(1000):
            puzzle = random.choice(puzzle.actions)[0]()
        return puzzle

    def copy(self):
        """
        Return a new puzzle with the same board as 'self'
        """
        board = []
        for row in self.board:
            board.append([x for x in row])
        return Puzzle(board)

    def _move(self, at, to):
        """
        Return a new puzzle where 'at' and 'to' tiles have been swapped.
        NOTE: all moves should be 'actions' that have been executed
        """
        copy = self.copy()
        i, j = at
        r, c = to
        copy.board[i][j], copy.board[r][c] = copy.board[r][c], copy.board[i][j]
        return copy

    def pprint(self):
        for row in self.board:
            print(row)
        print()

    def __str__(self):
        return ''.join(map(str, self))

    def __iter__(self):
        for row in self.board:
            yield from row


# example of use     
board = [[1,2,3],[4,5,0],[6,7,8]]
puzzle = Puzzle(board)
#puzzle = puzzle.shuffle()
s = Solver(puzzle)
p = s.solve()

steps = 0
for node in p:
    print(node.action)
    node.puzzle.pprint()
    steps += 1

print("Total number of steps: " + str(steps))

None
[1, 2, 3]
[4, 5, 0]
[6, 7, 8]

D
[1, 2, 3]
[4, 5, 8]
[6, 7, 0]

L
[1, 2, 3]
[4, 5, 8]
[6, 0, 7]

L
[1, 2, 3]
[4, 5, 8]
[0, 6, 7]

U
[1, 2, 3]
[0, 5, 8]
[4, 6, 7]

R
[1, 2, 3]
[5, 0, 8]
[4, 6, 7]

D
[1, 2, 3]
[5, 6, 8]
[4, 0, 7]

R
[1, 2, 3]
[5, 6, 8]
[4, 7, 0]

U
[1, 2, 3]
[5, 6, 0]
[4, 7, 8]

L
[1, 2, 3]
[5, 0, 6]
[4, 7, 8]

L
[1, 2, 3]
[0, 5, 6]
[4, 7, 8]

D
[1, 2, 3]
[4, 5, 6]
[0, 7, 8]

R
[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

R
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]

Total number of steps: 14


In [1]:
import random
import itertools
import collections
import time

class Node:
    def __init__(self, puzzle, parent=None, action=None):
        self.puzzle = puzzle
        self.parent = parent
        self.action = action
        self.g = parent.g + 1 if parent else 0

    @property
    def score(self):
        return self.g + self.h

    @property
    def state(self):
        return str(self)

    @property 
    def path(self):
        node, p = self, []
        while node:
            p.append(node)
            node = node.parent
        yield from reversed(p)

    @property
    def solved(self):
        return self.puzzle.solved

    @property
    def actions(self):
        return self.puzzle.actions

    @property
    def h(self):
        return self.puzzle.manhattan

    @property
    def f(self):
        return self.h + self.g

    def __str__(self):
        return str(self.puzzle)

class Solver:
    def __init__(self, start):
        self.start = start

    def solve(self):
        queue = collections.deque([Node(self.start)])
        seen = set()
        seen.add(queue[0].state)
        
        while queue:
            queue = collections.deque(sorted(list(queue), key=lambda node: node.f))
            node = queue.popleft()
            
            if node.solved:
                return node.path

            for move, action in node.actions:
                child = Node(move(), node, action)

                if child.state not in seen:
                    queue.appendleft(child)
                    seen.add(child.state)

class Puzzle:
    def __init__(self, board):
        self.width = len(board[0])
        self.board = board

    @property
    def solved(self):
        N = self.width * self.width
        return str(self) == ''.join(map(str, range(1, N))) + '0'

    @property 
    def actions(self):
        def create_move(at, to):
            return lambda: self._move(at, to)

        moves = []
        for i, j in itertools.product(range(self.width), range(self.width)):
            direcs = {'R':(i, j-1), 'L':(i, j+1), 'D':(i-1, j), 'U':(i+1, j)}
            for action, (r, c) in direcs.items():
                if r >= 0 and c >= 0 and r < self.width and c < self.width and self.board[r][c] == 0:
                    move = create_move((i, j), (r, c)), action
                    moves.append(move)
        return moves

    @property
    def manhattan(self):
        distance = 0
        for i in range(4):
            for j in range(4):
                if self.board[i][j] != 0:
                    x, y = divmod(self.board[i][j] - 1, 4)
                    distance += abs(x - i) + abs(y - j)
        return distance

    def shuffle(self):
        puzzle = self
        for _ in range(1000):
            puzzle = random.choice(puzzle.actions)[0]()
        return puzzle

    def copy(self):
        board = [row[:] for row in self.board]
        return Puzzle(board)

    def _move(self, at, to):
        copy = self.copy()
        i, j = at
        r, c = to
        copy.board[i][j], copy.board[r][c] = copy.board[r][c], copy.board[i][j]
        return copy

    def pprint(self):
        for row in self.board:
            print(row)
        print()

    def __str__(self):
        return ''.join(map(str, self))

    def __iter__(self):
        for row in self.board:
            yield from row


# Esempio di utilizzo con puzzle 4x4
board = [[1, 2, 15, 14], [11, 6, 7, 8], [9, 10, 5, 12], [3, 4, 0, 13]]
puzzle = Puzzle(board)
s = Solver(puzzle)
p = s.solve()

steps = 0
for node in p:
    print(node.action)
    node.puzzle.pprint()
    steps += 1

print("Total number of steps:", steps)


KeyboardInterrupt: 

In [ ]:
import random
import itertools
import collections

class Node:
    def __init__(self, puzzle, parent=None, action=None):
        self.puzzle = puzzle
        self.parent = parent
        self.action = action
        self.g = parent.g + 1 if parent else 0

    @property
    def score(self):
        return self.g + self.h

    @property
    def state(self):
        return str(self)

    @property 
    def path(self):
        node, p = self, []
        while node:
            p.append(node)
            node = node.parent
        yield from reversed(p)

    @property
    def solved(self):
        return self.puzzle.solved

    @property
    def actions(self):
        return self.puzzle.actions

    @property
    def h(self):
        return self.puzzle.manhattan

    @property
    def f(self):
        return self.h + self.g

    def __str__(self):
        return str(self.puzzle)

class Solver:
    def __init__(self, start):
        self.start = start

    def solve(self):
        queue = collections.deque([Node(self.start)])
        seen = set()
        seen.add(queue[0].state)
        
        while queue:
            queue = collections.deque(sorted(list(queue), key=lambda node: node.f))
            node = queue.popleft()
            
            if node.solved:
                return node.path

            for move, action in node.actions:
                child = Node(move(), node, action)

                if child.state not in seen:
                    queue.appendleft(child)
                    seen.add(child.state)

class Puzzle:
    def __init__(self, board):
        self.width = len(board[0])
        self.board = board

    @property
    def solved(self):
        N = self.width * self.width
        return str(self) == ''.join(map(str, range(1, N))) + '0'

    @property 
    def actions(self):
        def create_move(at, to):
            return lambda: self._move(at, to)

        moves = []
        for i, j in itertools.product(range(self.width), range(self.width)):
            direcs = {'R': (i, j - 1), 'L': (i, j + 1), 'D': (i - 1, j), 'U': (i + 1, j)}
            for action, (r, c) in direcs.items():
                if r >= 0 and c >= 0 and r < self.width and c < self.width and self.board[r][c] == 0:
                    move = create_move((i, j), (r, c)), action
                    moves.append(move)
        return moves

    @property
    def manhattan(self):
        distance = 0
        for i in range(5):
            for j in range(5):
                if self.board[i][j] != 0:
                    x, y = divmod(self.board[i][j] - 1, 5)
                    distance += abs(x - i) + abs(y - j)
        return distance

    def shuffle(self):
        puzzle = self
        for _ in range(1000):
            puzzle = random.choice(puzzle.actions)[0]()
        return puzzle

    def copy(self):
        board = [row[:] for row in self.board]
        return Puzzle(board)

    def _move(self, at, to):
        copy = self.copy()
        i, j = at
        r, c = to
        copy.board[i][j], copy.board[r][c] = copy.board[r][c], copy.board[i][j]
        return copy

    def pprint(self):
        for row in self.board:
            print(row)
        print()

    def __str__(self):
        return ''.join(map(str, self))

    def __iter__(self):
        for row in self.board:
            yield from row


# Esempio di utilizzo con puzzle 5x5
board = [
    [24, 2, 19, 4, 5],
    [6, 7, 8, 0, 10],
    [11, 12, 13, 14, 15],
    [16, 17, 18, 3, 20],
    [21, 22, 9, 23, 1]
]
puzzle = Puzzle(board)
s = Solver(puzzle)
p = s.solve()

steps = 0
for node in p:
    print(node.action)
    node.puzzle.pprint()
    steps += 1

print("Total number of steps:", steps)
